In [1]:
# allow to reload modules
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import argparse
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import SegformerForImageClassification, SegformerModel
import torch
from torch import nn

import sys

sys.path.append("..")

from core.datasets.build import transform
from core.configs import cfg
from core.models.segformer import *
from core.utils.hyperbolic import HyperMapper, HyperMLR

/home/pmandica/miniconda3/envs/openmmlab/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [3]:
import os


def parse_args(args_str: str = None):
    parser = argparse.ArgumentParser(description="Active Domain Adaptive Semantic Segmentation Training")
    parser.add_argument("-cfg",
                        "--config-file",
                        default="",
                        metavar="FILE",
                        help="path to config file",
                        type=str)
    parser.add_argument("--proctitle",
                        type=str,
                        default="HALO",
                        help="allow a process to change its title", )
    parser.add_argument(
        "opts",
        help="Modify config options using the command-line",
        default=None,
        nargs=argparse.REMAINDER
    )

    args_list = args_str.split() if args_str else None
    args = parser.parse_args(args_list)

    if args.opts is not None and args.opts != []:
        args.opts[-1] = args.opts[-1].strip('\r\n')

    cfg.set_new_allowed(True)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    cfg.SAVE_DIR = os.path.join(cfg.OUTPUT_DIR, cfg.NAME)
    print("Saving to {}".format(cfg.SAVE_DIR))
    cfg.freeze()

    return args

In [4]:
args = parse_args("-cfg ../configs/gtav/debug.yaml")
cfg

Saving to results/source_target/debug


CfgNode({'MODEL': CfgNode({'NAME': 'deeplabv3plus_resnet101', 'NUM_CLASSES': 19, 'WEIGHTS': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth', 'FREEZE_BN': True, 'HYPER': True, 'CURVATURE': 1.0, 'REDUCED_CHANNELS': 64, 'HFR': True}), 'WANDB': CfgNode({'ENABLE': False, 'GROUP': 'source_target', 'PROJECT': 'active_domain_adapt', 'ENTITY': 'pinlab-sapienza'}), 'INPUT': CfgNode({'SOURCE_INPUT_SIZE_TRAIN': (1280, 720), 'TARGET_INPUT_SIZE_TRAIN': (1280, 640), 'INPUT_SIZE_TEST': (1280, 640), 'INPUT_SCALES_TRAIN': (1.0, 1.0), 'IGNORE_LABEL': 255, 'PIXEL_MEAN': [0.485, 0.456, 0.406], 'PIXEL_STD': [0.229, 0.224, 0.225], 'TO_BGR255': False}), 'DATASETS': CfgNode({'SOURCE_TRAIN': 'gtav_train', 'TARGET_TRAIN': 'cityscapes_train', 'TEST': 'cityscapes_val'}), 'SOLVER': CfgNode({'GPUS': [0], 'NUM_ITER': 60000, 'LR_METHOD': 'poly', 'BASE_LR': 0.001, 'LR_POWER': 0.5, 'MOMENTUM': 0.9, 'WEIGHT_DECAY': 0.0005, 'WARMUP_ITERS': 600, 'BATCH_SIZE': 2, 'BATCH_SIZE_VAL': 1, 'CONSISTENT_LOSS': 0.0, 'NE

In [5]:
from core.datasets.build import build_dataset
from core.datasets.cityscapes import cityscapesDataSet

w, h = cfg.INPUT.TARGET_INPUT_SIZE_TRAIN
trans_list = [
    transform.ToTensor(),
    transform.Normalize(mean=cfg.INPUT.PIXEL_MEAN, std=cfg.INPUT.PIXEL_STD, to_bgr255=cfg.INPUT.TO_BGR255)
]
if cfg.INPUT.INPUT_SCALES_TRAIN[0] == cfg.INPUT.INPUT_SCALES_TRAIN[1] and cfg.INPUT.INPUT_SCALES_TRAIN[0] == 1:
    trans_list = [transform.Resize((h, w)), ] + trans_list
else:
    trans_list = [
                        transform.RandomScale(scale=cfg.INPUT.INPUT_SCALES_TRAIN),
                        transform.RandomCrop(size=(h, w), pad_if_needed=True),
                    ] + trans_list
trans = transform.Compose(trans_list)

target_set = cityscapesDataSet(
                "../datasets/cityscapes",
                "../datasets/cityscapes_train_list.txt",
                max_iters=1000,
                num_classes=19,
                split="train",
                transform=trans,
                cfg=cfg,
                empty=False,
            )

target_loader = DataLoader(
            dataset=target_set,
            batch_size=2,
            shuffle=True,
            num_workers=4,
            pin_memory=True,
            drop_last=True,
            persistent_workers=True,)

In [6]:
batch = next(iter(target_loader))

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/pmandica/miniconda3/envs/openmmlab/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/pmandica/miniconda3/envs/openmmlab/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/pmandica/miniconda3/envs/openmmlab/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/pmandica/HALO/analysis/../core/datasets/cityscapes.py", line 234, in __getitem__
    label_mask = np.array(Image.open(datafiles["label_mask"]), dtype=np.uint8)
  File "/home/pmandica/miniconda3/envs/openmmlab/lib/python3.8/site-packages/PIL/Image.py", line 3227, in open
    fp = builtins.open(filena

In [7]:
img, label = batch['img'], batch['label']
img.shape

torch.Size([2, 3, 640, 1280])

In [7]:
img = torch.rand(2, 3, 720, 1280)

In [3]:
feature_extractor = SegformerModel.from_pretrained("nvidia/mit-b2")

In [10]:
config = feature_extractor.config
config.num_labels = 19
return_dict = config.use_return_dict

In [11]:
outputs = feature_extractor(img, output_hidden_states=True, return_dict=return_dict)

In [12]:
encoder_hidden_states = outputs.hidden_states if return_dict else outputs[1]

In [13]:
head = SegformerDecodeHead(config)
head.post_init()

In [20]:
outputs[1][1].shape

torch.Size([2, 128, 90, 160])

In [44]:
out = head(encoder_hidden_states)

In [45]:
out.shape

torch.Size([2, 19, 160, 320])

In [7]:
from torch.optim import Adam

optimizer1 = Adam(feature_extractor.parameters(), lr=1e-4)
optimizer2 = Adam(feature_extractor.parameters(), lr=1e-5)
optims = [optimizer1, optimizer2]

In [8]:
print(optims)

[Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.0001
    maximize: False
    weight_decay: 0
), Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 1e-05
    maximize: False
    weight_decay: 0
)]
